# I/O

## 1. Input from the code

In [22]:
a = input('Please, enter something: ')
print('You entered:', a)

Please, enter something: dfas
You entered: dfas


## 2. Input from the OS prompt

In [17]:
! cat argparse_example.py

import argparse

parser = argparse.ArgumentParser(description='This program bla, bla, bla ...')
parser.add_argument('-i', '--input', help='Input file name', default='stdin')

args = parser.parse_known_args()[0]

#parser.add_argument('-o', '--output', help='Output file name', default='stdout')
#requiredNamed = parser.add_argument_group('required named arguments')
#requiredNamed.add_argument('-i', '--input', help='Input file name', required=True)

#parser.parse_args(['-h'])


In [20]:
! python3 argparse_example.py -h

usage: argparse_example.py [-h] [-i INPUT]

This program bla, bla, bla ...

optional arguments:
  -h, --help            show this help message and exit
  -i INPUT, --input INPUT
                        Input file name


In [21]:
! python3 argparse_example.py -i abc

Using input file abc


## 3. Disk

In [23]:
import io

Write some integers:

In [24]:
# The legacy way:
file = open('/tmp/some_integers_1.txt', 'w')
file.write('{}\n'.format(1))
file.write('{}\n'.format(2))
file.write('{}\n'.format(3))
file.close()

In [25]:
! cat /tmp/some_integers_1.txt

1
2
3


In [26]:
# The modern (pythonic) alternative:
with io.open('/tmp/some_integers_2.txt', 'w') as file:
    file.write('{}\n'.format(1))
    file.write('{}\n'.format(2))
    file.write('{}\n'.format(3))

In [27]:
file.closed

True

In [28]:
! cat /tmp/some_integers_2.txt

1
2
3


Reading the file:

In [29]:
# The classic alternative:
file = io.open('/tmp/some_integers_1.txt', 'r')
while True:
    line = file.readline()
    if not line:
        break
#for i in range(3):
    print(int(line))
file.close()

1
2
3


In [30]:
# The pythonic alternarive:
with io.open('/tmp/some_integers_2.txt', 'r') as file:
    for line in file:
        print(int(line))

1
2
3


In [31]:
file.closed

True

## 4. Network

Using TCP:

In [32]:
import socket

PORT = 8001

class TCP_Receiver():

    # We use a context manager
    # (https://docs.python.org/3/reference/datamodel.html#context-managers).
    def __enter__(self):
        '''Create a TCP socket.'''
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.sock.bind(('', PORT))
        self.sock.listen(1)
        print('waiting for a connection at {} ...'\
              .format(self.sock.getsockname()))
        return self

    def accept(self):
        (self.peer_sock, peer) = self.sock.accept()
        print('\nconnection accepted from {}.'.format(peer))

    def receive(self):
        message_final_length = len('hello world!')
        message = bytearray() # bytearray is mutable and therefore, faster appending than types
        while len(message) < message_final_length:
            chunk = self.peer_sock.recv(message_final_length - len(message))
            if chunk == b'':
                raise RuntimeError("socket connection broken")
            print('received "{}"'.format(chunk))
            message.extend(chunk)
        return message
    
    def __exit__(self,ext_type,exc_value,traceback):
        self.sock.close()
        print('socket closed')

def use_socket():
    with TCP_Receiver() as recv:
        recv.accept()
        print('message = {}'.format(recv.receive()))
        
import threading

threading.Thread(target=use_socket).start()

waiting for a connection at ('0.0.0.0', 8001) ...


In [33]:
# Create a TCP socket 
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((socket.gethostname(), PORT))
print('connection established with {}'.format((sock.getpeername())))

message = b'hello world!'
total_sent = 0 # bytes
while total_sent < len(message):
    sent = sock.send(message[total_sent:])
    if sent == 0:
        raise RuntimeError("socket connection broken")
    total_sent += sent
sock.close()

gaierror: [Errno 8] nodename nor servname provided, or not known

Using UDP:

In [8]:
import socket

PORT = 8001

class UDP_Receiver():

    # We use a context manager (https://docs.python.org/3/reference/datamodel.html#context-managers).
    def __enter__(self):
        '''Create a TCP socket.'''
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.sock.bind(('', PORT))
        print('listening at {} ...'\
              .format(self.sock.getsockname()))
        return self

    def receive(self):
        (message, from_addr) = self.sock.recvfrom(1024)
        print('received {} from {}'.format(message, from_addr))
        return message
    
    def __exit__(self,ext_type,exc_value,traceback):
        self.sock.close()
        print('socket closed')

def use_socket():
    with UDP_Receiver() as recv:
        print('message = {}'.format(recv.receive()))
        
import threading

threading.Thread(target=use_socket).start()

listening at ('0.0.0.0', 8001) ...


In [9]:
# Create a UDP socket 
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
message = b'hello world!'
sock.sendto(message, (socket.gethostname(), PORT))
sock.close()

received b'hello world!' from ('192.168.1.133', 58840)
message = b'hello world!'
socket closed
